In [ ]:
# Work 12: Pivoting and Summarizing ICD-10 Diagnoses by Patient_ID [W12.HFRS.6.Pivot_HFRS.ipynb]

# "This Jupyter notebook script organizes ICD-10 diagnoses by patient ID, truncates codes, pivots data, and
#  summarizes code lengths for analysis."

########################################################################################################
#  Sequence list
########################################################################################################

#  1. Create a function that truncates the ICD-10 code to the first five characters and removes the dot.
#  2. Load data from the file (all_data_path).
#  3. Truncate ICD-10 codes to five characters using the created function.
#  4. Filter out empty ICD-10 codes before counting.
#  5. Calculate the length of each ICD-10 code from the filtered data and generate summaries.
#  6. Calculate how many ICD-10 codes fall into each length category.
#  7. Calculate the total sum for all categories.
#  8. Print a summary of ICD-10 code lengths and the total sum.
#  9. Print the summary again (ICD-10 code lengths).
# 10. Number each patient's ICD codes using a cumulative counter.
# 11. Pivot the data by ICD codes so each row is a patient with ICD codes in separate columns.
# 12. Fill any empty cells after pivoting (e.g., with an empty string or NaN).
# 13. Rename the columns (e.g., 'code_1', 'code_2', etc.).
# 14. Reset the index so that "Potilas_ID" is a column again.
# 15. Arrange the ICD codes numerically after pivoting.
# 15.1 Remove rows where all ICD code columns are empty.
# 16. Calculate summaries of ICD code lengths after pivoting.
# 17. Melt the pivot table back into long format to facilitate summary creation.
# 18. Filter out empty ICD-10 codes from the melted data.
# 19. Calculate code lengths and generate a summary of the filtered codes.
# 20. Calculate the total sum for all length categories after pivoting.
# 21. Print a summary of ICD-10 code lengths after pivoting and the total sum.
# 22. Save the transformed data to a CSV file.
# 23. Calculate the difference in the number of ICD-10 codes before and after pivoting.
# 24. Print the difference in the number of ICD-10 codes before and after pivoting.

########################################################################################################
########################################################################################################

import pandas as pd


# 1: Create a function to truncate the code into the first 5 characters and remove the dot
def truncate_icd10_code(icd_code):
    """Truncate the ICD-10 code to the first 5 characters and remove the dot."""
    truncated_code = icd_code[:5]  # First letter and first 4 digits
    return truncated_code.replace(".", "")  # Remove the dot


print("1: Created a function to truncate ICD-10 code")

# 2: Create paths and load data
all_data_path = "/home/work/all_data.csv"
all_data_df = pd.read_csv(all_data_path, dtype=str, usecols=["Potilas_ID", "ICD_code"])
transformed_data_path = "/home/work/pivoted_all_data.csv"

print("2: Data loaded successfully.")

# 3: Truncate the ICD-10 codes
all_data_df["ICD_code"] = all_data_df["ICD_code"].apply(truncate_icd10_code)

print("3: ICD codes truncated successfully.")

# 4: Filter out empty ICD-10 codes before counting
all_data_df = all_data_df[all_data_df["ICD_code"].notna() & (all_data_df["ICD_code"] != "")]

print("4: Filtered out empty ICD-10 codes before counting.")

# 5: Calculate the length of each ICD-10 code and generate summaries from the filtered data
all_data_df["code_length"] = all_data_df["ICD_code"].str.len()

print(
    "5: Calculated the length of each ICD-10 code and generated summaries from the filtered data."
)

# 6: Calculate how many ICD-10 codes fall into each length category
code_length_summary = all_data_df["code_length"].value_counts().sort_index()

print("6: Calculated how many ICD-10 codes fall into each length category.")

# 7: Calculate the total sum for all categories
total_icd_codes = code_length_summary.sum()

print("7: Calculated the total sum for all categories.")

# 8: Print summary and total sum
print("8A: Summary of ICD-10 code lengths:")
print(code_length_summary)
print(f"8B: Total for all ICD-10 codes: {total_icd_codes}")

# 9: Print the summary again
print("9: Summary of ICD-10 code lengths:")
print(code_length_summary)

# 10: Number each patient's ICD codes
all_data_df["ICD_code_num"] = all_data_df.groupby("Potilas_ID").cumcount() + 1

print("10: Numbered each patient's ICD codes.")

# 11: Pivot the data by ICD codes
all_data_pivot = all_data_df.pivot(index="Potilas_ID", columns="ICD_code_num", values="ICD_code")

print("11: Pivoted the data by ICD codes.")

# 12: Fill any empty cells (e.g., with an empty string or NaN)
all_data_pivot = all_data_pivot.fillna("")

print("12: Filled any empty cells.")

# 13: Rename the columns
all_data_pivot.columns = [f"code_{int(col)}" for col in all_data_pivot.columns]

print("13: Renamed the columns.")

# 14: Reset the index so that "Potilas_ID" becomes a column again
all_data_pivot.reset_index(inplace=True)

print("14: Data transformed successfully.")
print(all_data_pivot.head())

# 15: Arrange ICD codes numerically after pivoting
all_data_pivot = all_data_pivot.reindex(
    sorted(
        all_data_pivot.columns,
        key=lambda x: (int(x.split("_")[1]) if "code_" in x else float("inf")),
    ),
    axis=1,
)

# 15.1: Remove rows where all ICD code columns are empty
icd_code_columns = [col for col in all_data_pivot.columns if col.startswith("code_")]
all_data_pivot = all_data_pivot.dropna(subset=icd_code_columns, how="all")

print("15: Arranged ICD codes numerically after pivoting and removed empty rows.")

# 16: Calculate summaries of ICD code lengths after pivoting
# After pivoting, each ICD code is in its own column
# Here we calculate the character lengths of each column and combine them into a summary
icd_code_columns = [col for col in all_data_pivot.columns if col.startswith("code_")]

print("16: Calculated summaries of ICD code lengths after pivoting.")

# 17: Melt the pivot table back into a long format for creating summaries
melted_df = all_data_pivot.melt(
    id_vars=["Potilas_ID"], value_vars=icd_code_columns, var_name="code", value_name="ICD_code"
)

print("17: Melted the pivot table back into a long format for summary creation.")

# 18: Filter out empty ICD-10 codes
melted_df = melted_df[melted_df["ICD_code"].notna() & (melted_df["ICD_code"] != "")]

print("18: Filtered out empty ICD-10 codes.")

# 19: Calculate lengths and create a summary of the filtered codes
melted_df["code_length"] = melted_df["ICD_code"].str.len()
code_length_summary_pivot = melted_df["code_length"].value_counts().sort_index()

print("19: Calculated lengths and created a summary of the filtered codes.")

# 20: Calculate the total sum for all length categories after pivoting
total_icd_codes_pivot = code_length_summary_pivot.sum()

print("20: Calculated the total sum for all length categories after pivoting.")

# 21: Print the summary and the total sum after pivoting
print("21A: Summary of ICD-10 code lengths after pivoting:")
print(code_length_summary_pivot)
print(f"21B: Total for all ICD-10 codes after pivoting: {total_icd_codes_pivot}")

# 22: Save the transformed data to a CSV file
all_data_pivot.to_csv(transformed_data_path, index=False)

print(f"22: Transformed data saved to {transformed_data_path}.")

# 23: Calculate the difference in the number of ICD-10 codes before and after pivoting
icd_code_difference = total_icd_codes - total_icd_codes_pivot

# 24: Print the difference
print(
    f"23-24: Difference in the number of ICD-10 codes before and after pivoting: {icd_code_difference}"
)


########################################################################################################
########################################################################################################